In [32]:
import pandas
import plotly.graph_objects as go
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,Dropdown
import os
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px



In [33]:
exps_base = 'exps'
exps = sorted([x for x in os.listdir(os.path.expanduser('../' + exps_base)) if 'DS_Store' not in x and '.csv' not in x])
exp_folder = None
selected_timestamp = None


def timestamp_select(timestamp):
    global selected_timestamp
    selected_timestamp = timestamp
    
    
def exp_select(exp_name):
    global exp_folder 
    exp_folder = exp_name
    timestamps = sorted([x for x in os.listdir(os.path.expanduser('../{0}/{1}'.format(exps_base,exp_folder))) if 'DS_Store' not in x],reverse = True)
    interact(timestamp_select, timestamp=timestamps);



def get_img(img,is_label):
    if (is_label):
        img = Image.fromarray(img.squeeze().astype(np.uint8) * 100, 'L')
    else:
        img = Image.fromarray(img.squeeze().astype(np.uint8))
    return img


In [ ]:
button = widgets.Button(description="reload")
from IPython.display import display
from IPython.display import clear_output
with_step = True
display(button)

def plot_loss():
    global selected_stepdata
    global selected_data
    trace_steploss = []
    trace_loss =[]
    eps_trace = []
    trace_loss = None
    trace_effective_distance = None

    if (with_step and 'loss_step' in list(selected_stepdata.columns) ):
        trace_steploss = [go.Scatter(x=selected_stepdata['Unnamed: 0'],y=selected_stepdata.loss_step,mode='lines',name='train_loss')]
    
    if (with_step and 'margin_loss_clamped_mean' in list(selected_stepdata.columns)):
        trace_steploss = trace_steploss + [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                                      y=selected_stepdata.margin_loss_clamped_mean,
                                                      mode='lines',name='margin_loss_clamped_mean')]
        
    if (with_step and 'correct_points_loss_sum' in list(selected_stepdata.columns)):
        trace_steploss = trace_steploss + [go.Scatter(x=selected_stepdata['Unnamed: 0'],y=selected_stepdata.correct_points_loss_sum,mode='lines',name='correct_points_loss_sum'),
                      go.Scatter(x=selected_stepdata['Unnamed: 0'],y=selected_stepdata.wrong_points_loss_sum,mode='lines',name='wrong_points_loss_sum')                     ]
                      
    if ('loss_epoch' in list(selected_data.columns)):
        trace_loss = [go.Scatter(x=selected_data.epoch,y=selected_data.loss_epoch,mode='lines',name='train_loss')]
        
        trace_train_accuracy = go.Scatter(x=selected_data.epoch,
                                          y=selected_data.train_accuracy,
                                          mode='lines+markers',
                                          name='train_accuracy')
    
        trace_lr = go.Scatter(x=selected_data.epoch,y=selected_data.lr_epoch,mode='lines',name='lr')
    if ('correct_real_mean_loss_log' in list(selected_data.columns)):        
        trace_loss = trace_loss + [go.Scatter(x=selected_data.epoch,y=selected_data.correct_real_mean_loss_log,mode='lines',name='correct_real_mean_loss_log'),
                                      go.Scatter(x=selected_data.epoch,y=selected_data.correct_mean_loss_log,mode='lines',name='correct_mean_loss_log')]
        
    if ('batch_margin_mean_loss_log' in list(selected_data.columns)):        
        trace_loss = trace_loss + [go.Scatter(x=selected_data.epoch,y=selected_data.batch_margin_mean_loss_log,mode='lines',name='margin_mean_loss')]
    
    if ('effective_distance_log' in list(selected_data.columns)):
        trace_effective_distance = go.Scatter(x=selected_data.epoch,y=selected_data.effective_distance_log,mode='lines',name='effective_distance_log')
    
    trace_test_accuracy = go.Scatter(x=selected_data.epoch,
                                     y=selected_data.test_accuracy,
                                     mode='lines+markers',
                                     name='test_accuracy',connectgaps=True)
    pgd_cols = [x for x in list(selected_data.columns) if 'pgd' in x]
    pgd_traces = [go.Scatter(x=selected_data.epoch,y=selected_data[col],
                             mode='lines+markers',connectgaps=True) 
                  for col in pgd_cols]

    fig = make_subplots(rows=5 + len(pgd_cols), cols=1,
                        subplot_titles=["steploss",
                                        "loss",
                                        "accuracy_interpolate",
                                       "effective distance"]
                                        + pgd_cols  + ["lr"])
    
    [fig.add_trace(t,row=1,col=1) for t in trace_steploss]
    if len (trace_loss) >0:
        [fig.add_trace(t,row=2,col=1) for t in trace_loss ]
        fig.add_trace(trace_train_accuracy,row=3,col=1)
        fig.add_trace(trace_lr,row=5 + len(pgd_cols),col=1)
    fig.add_trace(trace_test_accuracy,row=3,col=1)
    
    if (trace_effective_distance):
        fig.add_trace(trace_effective_distance,row=4,col=1)
    
    for i,tr in enumerate(pgd_traces):
        fig.add_trace(tr,row=5+i,col=1)
                  
    
    fig.update_layout(
    autosize=False,
    width=800,
    height=2000)
    fig.show()
    
    correct_effective_trace = []
    if (with_step and 'correct_count' in list(selected_stepdata.columns)):
        correct_effective_trace = [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                            y=selected_stepdata.correct_number_of_pgd_effective/selected_stepdata.correct_count ,mode='lines',
                           name='correct_effective_pgd') ]
    if (with_step and 'wrong_number_of_pgd_effective' in list(selected_stepdata.columns)):
        correct_effective_trace += [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                            y=selected_stepdata.wrong_number_of_pgd_effective/selected_stepdata.wrong_count ,mode='lines',
                           name='wrong_effective_pgd') ]
    
    
    if (with_step and 'correct_number_of_original_pgd_effective' in list(selected_stepdata.columns)):
        correct_effective_trace = correct_effective_trace + [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                            y=selected_stepdata.correct_number_of_original_pgd_effective/selected_stepdata.correct_count ,mode='lines',
                           name='correct_number_of_original_pgd_effective') ]
        
    

    correct_distance_trace = []
    if ( with_step and 'correct_effective_zero_points_distance_mean' in list(selected_stepdata.columns)):
        correct_distance_trace.append(go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_effective_zero_points_distance_mean ,mode='lines',
                                name='correct_distance_trace') )

   
    if (with_step and 'correct_effective_zero_points_distance_star_mean' in list(selected_stepdata.columns)):
        correct_distance_trace.append(go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_effective_zero_points_distance_star_mean ,mode='lines',
                                name='correct_distance_star_trace'))
    if (with_step and 'correct_effective_zero_points_distance_max' in list(selected_stepdata.columns)):
        correct_distance_trace.append(go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_effective_zero_points_distance_max ,mode='lines',
                                name='correct_effective_zero_points_distance_max'))
      
    
    correct_zero_points_eval_trace = []
    if (with_step and 'correct_zero_points_eval_max' in list(selected_stepdata.columns)):
        correct_zero_points_eval_trace = [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_zero_points_eval_max ,mode='lines',
                               name='correct_zero_points_eval_max'),
                                go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_zero_points_eval_mean ,mode='lines',
                               name='correct_zero_points_eval_mean')]
    
    if (with_step and 'correct_sample_network_zero_points_eval_mean' in list(selected_stepdata.columns)):
        correct_zero_points_eval_trace =correct_zero_points_eval_trace + [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_sample_network_zero_points_eval_max ,mode='lines',
                               name='correct_sample_network_zero_points_eval_max'),
                                go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_sample_network_zero_points_eval_mean ,mode='lines',
                               name='correct_sample_network_zero_points_eval_mean')]

    correct_zero_points_grad_trace = []
    if (with_step and 'correct_zero_points_grad_norm_max' in list(selected_stepdata.columns)):
        correct_zero_points_grad_trace = [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_zero_points_grad_norm_max ,mode='lines',
                               name='correct_zero_points_grad_norm_max'),
                                go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_zero_points_grad_norm_mean ,mode='lines',
                               name='correct_zero_points_grad_norm_mean'),
                                         go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.correct_zero_points_grad_norm_min ,mode='lines',
                               name='correct_zero_points_grad_norm_min')]
    if (with_step and 'eps_correct' in list(selected_stepdata.columns)):
        eps_trace = [go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.eps_correct ,mode='lines',
                               name='eps_correct'),
                                go.Scatter(x=selected_stepdata['Unnamed: 0'],
                                y=selected_stepdata.eps_wrong ,mode='lines',
                               name='eps_wrong')]
        


    fig = make_subplots(rows=5, cols=1,subplot_titles=['correct_effective_pgd',
                                                       'correct_effective_distance',
                                                       'correct_zero_points_eval',
                                                      'correct_zero_points_grad',
                                                      'epsilon'])
    [fig.add_trace(t,row=1,col=1) for t in correct_effective_trace]
    [fig.add_trace(t,row=2,col=1) for t in correct_distance_trace]
    [fig.add_trace(t,row=3,col=1) for t in correct_zero_points_eval_trace]
    [fig.add_trace(t,row=4,col=1) for t in correct_zero_points_grad_trace]
    [fig.add_trace(t,row=5,col=1) for t in eps_trace]


    fig.update_layout(
    autosize=False,
    width=800,
    height=800)
    fig.show()



    
def on_button_clicked(b):
    clear_output()
    global selected_data
    global selected_stepdata
    global debug_conf
    
    if (os.path.isfile(os.path.join('../{0}'.format(exps_base),
                                                       exp_folder,
                                                       selected_timestamp,
                                                      'log',
                                                      'debug_conf.csv'))):
        debug_conf = pd.read_csv(os.path.join('../{0}'.format(exps_base),
                                                           exp_folder,
                                                           selected_timestamp,
                                                          'log',
                                                          'debug_conf.csv'))
        pd.set_option('max_colwidth', 400)

        display(debug_conf.T)
        
        
    if (os.path.isfile(os.path.join('../{0}'.format(exps_base),
                                                       exp_folder,
                                                       selected_timestamp,
                                                      'log',
                                                      'learning_epoch_log.csv'))):
        selected_data = pd.read_csv(os.path.join('../{0}'.format(exps_base),
                                                           exp_folder,
                                                           selected_timestamp,
                                                          'log',
                                                          'learning_epoch_log.csv'))
                                    
        if (os.path.isfile(os.path.join('../{0}'.format(exps_base),
                                                       exp_folder,
                                                       selected_timestamp,
                                                      'log',
                                                      'learning_epoch_log.csv'))):
            
            selected_stepdata = pd.read_csv(os.path.join('../{0}'.format(exps_base),
                                                           exp_folder,
                                                           selected_timestamp,
                                                          'log',
                                                          'learning_step_log.csv'))
        plot_loss()
    else:
        print ("file exp does not exists")
    display(button)
    display(selected_data[[x for x in list(selected_data.columns) if 'acc' in x] + ['epoch']].max())
button.on_click(on_button_clicked)

In [31]:
interact(exp_select, exp_name=exps);


interactive(children=(Dropdown(description='exp_name', options=('cifa_xnet', 'cifar_ours_debug', 'cifar_ours_e…